In [ ]:
from tidygrad.tensor import Tensor
from tidygrad.functional import Embedding, embedding
import numpy as np
from lovely_numpy import Lo


from transformers import GPT2Tokenizer

In [ ]:
from safetensors import safe_open

In [ ]:
text = "In a hole in the ground there lived a"
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# tokens = tokenizer.encode(text)  # returns a list of integers
# tokens = Tensor(tokens)

tokens = [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ]

In [ ]:
tokens

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
model = safe_open("model.safetensors", framework="np")

In [ ]:
model.keys()[-10:]

['h.9.ln_2.bias',
 'h.9.ln_2.weight',
 'h.9.mlp.c_fc.bias',
 'h.9.mlp.c_fc.weight',
 'h.9.mlp.c_proj.bias',
 'h.9.mlp.c_proj.weight',
 'ln_f.bias',
 'ln_f.weight',
 'wpe.weight',
 'wte.weight']

In [ ]:
wte = Tensor(model.get_tensor("wte.weight"))
wpe = Tensor(model.get_tensor("wpe.weight"))

print(wpe)
print(wte)

Tensor[1024, 768](name="?" op=Load):
    v=array[1024, 768] f32 n=786432 (3Mb) x∈[-4.538, 4.065] μ=-0.001 σ=0.123
    
Tensor[50257, 768](name="?" op=Load):
    v=array[50257, 768] f32 n=38597376 (0.1Gb) x∈[-1.270, 1.785] μ=0.000 σ=0.144
    


In [ ]:
import tidygrad

In [ ]:
token_embeddings = embedding(wte, tokens)

positions = np.arange(len(tokens))
position_embeddings = embedding(wpe, positions)

embeddings = token_embeddings + position_embeddings
Lo(embeddings)

Tensor[10, 768](name="(embedding(?)+embedding(?))" op=Add):
    v=array[10, 768] f32 n=7680 (30Kb) x∈[-4.511, 3.938] μ=-9.411e-05 σ=0.219
    

In [ ]:
ln_1_w = model.get_tensor("h.0.ln_1.weight")
ln_1_b = model.get_tensor("h.0.ln_1.bias")

In [ ]:
a = Tensor([[2,2],[1,1]])
a

Tensor[2, 2](name="?" op=Load):
    v=array[2, 2] i64 n=4 x∈[1, 2] μ=1.500 σ=0.500 [[2, 2], [1, 1]]
    

In [ ]:
data = np.random.randn(100,50)

a = Tensor(data)

a.std(axis=None, keepdims=True), Lo(np.std(data, axis=None, ddof=1))

(Tensor[1, 1](name="pow((sum(var)/?),0.5)" op=Pow):
     v=array[1, 1] [[0.991]]
     ,
 0.991)

In [ ]:
def layer_norm(x,  w, b, eps=1e-8):
    mu = x.mean(axis=-1, keepdims=True)
    sigma = x.std(axis=-1, keepdims=True)

    return ((x - mu) / (sigma + eps) ) * w + b#  tensor[10, 768] n=7680 (30Kb) x∈[-0.788, 0.579] μ=-0.005 σ=0.106


In [ ]:
ln_1 = layer_norm(embeddings, ln_1_w, ln_1_b)
ln_1

#  tensor[10, 768] n=7680 (30Kb) x∈[-0.788, 0.579] μ=-0.005 σ=0.106

Tensor[10, 768](name="(((((embedding(?)+embedding(?))-(sum((embedding(?)+embedding(?)))/?))/(pow((sum(var)/?),0.5)+?))*?)+?)" op=Add):
    v=array[10, 768] f32 n=7680 (30Kb) x∈[-0.787, 0.579] μ=-0.005 σ=0.106
    